In [8]:
import sys
sys.path.append("../")
import warnings
import pickle
from pathlib import Path
from wsee.utils import utils
from wsee.data import pipeline
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [9]:
warnings.filterwarnings(action='once')

DATA_DIR = '/Users/phuc/data/snorkel-daystreamv6-end_loc_fix'  # replace path to corpus

In [10]:
loaded_data = pipeline.load_data(DATA_DIR)
sd_train = loaded_data['train']
sd_dev = loaded_data['dev']
sd_test = loaded_data['test']

daystream = loaded_data['daystream']

INFO:root:Reading train data from: /Users/phuc/data/snorkel-daystreamv6-end_loc_fix/train/train_with_events_and_defaults.jsonl
INFO:root:Reading dev data from: /Users/phuc/data/snorkel-daystreamv6-end_loc_fix/dev/dev_with_events_and_defaults.jsonl
INFO:root:Reading test data from: /Users/phuc/data/snorkel-daystreamv6-end_loc_fix/test/test_with_events_and_defaults.jsonl
INFO:root:Reading daystream data from: /Users/phuc/data/snorkel-daystreamv6-end_loc_fix/daystream.jsonl


In [11]:
merged_event_trigger_examples = pd.read_json(DATA_DIR + '/save_daystreamv6_triggers.jsonl', lines=True, encoding='utf8')
merged_event_role_examples = pd.read_json(DATA_DIR + '/save_daystreamv6_roles.jsonl', lines=True, encoding='utf8')

In [12]:
merged_examples: pd.DataFrame = utils.get_deep_copy(daystream)
for idx, row in merged_examples.iterrows():
    row['event_triggers'] = []
    row['event_roles'] = []
if 'id' in merged_examples:
    merged_examples.set_index('id', inplace=True)

merged_examples.update(merged_event_trigger_examples.set_index('id'))
merged_examples.update(merged_event_role_examples.set_index('id'))
len(merged_examples[merged_examples['event_triggers'].map(len) > 0])
merged_examples.reset_index(level=0, inplace=True)
# Removes rows with no events
merged_examples = merged_examples[merged_examples['event_triggers'].map(lambda d: len(d)) > 0]

merged_examples.to_json(DATA_DIR + '/daystream_snorkeled.jsonl', orient='records', lines=True, force_ascii=False)